In [1]:
import pandas as pd
import numpy as np
### requires openpyxl and mlxtend, install with conda
### may require restarting the kernel

[OnlineRetail.xlsx](https://drive.google.com/drive/folders/1_QhgQcyHsJXWJ1lycSUDstMIzf0UX1ka?usp=sharing)

In [2]:
data = pd.read_excel('OnlineRetail.xlsx')

In [3]:
from mlxtend.frequent_patterns import apriori, association_rules

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [6]:
data.Country.value_counts()

United Kingdom          495478
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
USA                        291
Hong Kong                  288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon 

### Identify number of customers

In [7]:
len(data.CustomerID.unique())

4373

### Identify customer doing most purchasing & amount

In [8]:
data['TotalPrice'] = data['Quantity'] * data['UnitPrice']

In [9]:
res = data.groupby(['CustomerID']).TotalPrice.sum()

In [10]:
res.sort_values(ascending=False)

CustomerID
14646.0    279489.02
18102.0    256438.49
17450.0    187482.17
14911.0    132572.62
12415.0    123725.45
             ...    
12503.0     -1126.00
17603.0     -1165.30
14213.0     -1192.20
15369.0     -1592.49
17448.0     -4287.63
Name: TotalPrice, Length: 4372, dtype: float64

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
 8   TotalPrice   541909 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 37.2+ MB


### Clean Data

In [12]:
data.InvoiceNo.value_counts()

573585     1114
581219      749
581492      731
580729      721
558475      705
           ... 
554023        1
554022        1
554021        1
554020        1
C558901       1
Name: InvoiceNo, Length: 25900, dtype: int64

### Finding all the credit records

In [ ]:
data[data.InvoiceNo.astype('str').str.startswith('C')]

In [ ]:
data = data[~data.InvoiceNo.astype('str').str.startswith('C')]

In [ ]:
data['Description'] = data.Description.str.strip()

### Basket Creation

In [ ]:
#german sales transactions
data_Germany = data[data.Country == 'Germany']

In [ ]:
data_Germany.groupby(['InvoiceNo','Description'])['Quantity'].sum()
#get qualities with InvoiceNo and Description as attributes

In [ ]:
data.Description.value_counts()

In [ ]:
basket_Germany = data[data['Country'] =="Germany"].groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')

In [ ]:
basket_Germany.head()

In [ ]:
basket_encoded = basket_Germany.applymap(lambda x: 0 if x <=0 else 1) 
basket_Germany = basket_encoded

In [ ]:
basket_Germany.shape

In [ ]:
frq_items = apriori(basket_Germany, min_support = 0.05, use_colnames = True) 

In [ ]:
rules = association_rules(frq_items, metric ="confidence", min_threshold = .1) 
# rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
# print(rules.head()) 

In [ ]:
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

In [ ]:
rules.head(20)

In [ ]:
rules['name'] = rules.antecedents.astype('str')

In [ ]:
rules[rules.name.str.contains('PLASTERS IN TIN SPACEBOY')]